In [2]:
import read_data
data = read_data.read_data(50)

/home/lo/pg/ML/ML-Projekt/read_data.py:30: RuntimeWarning: divide by zero encountered in log10
  stft = 10 * np.log10(stft)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

2024-01-10 20:26:19.278888: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 20:26:19.768325: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 20:26:19.781824: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 20:26:21.780358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Die Daten und Targets werden in numpy.Arrays mit den richtigen Shapes geschrieben 

X = np.concatenate(data['arr'].values).reshape(-1, 256, 128)
print(X.shape)

# Resnet braucht 3 Farbkanäle. Wir haben aber nur einen. Also gleicher Wert in alle 3 Kanäle
X = np.stack((X,) *3, axis=-1)
print(X.shape)

y = data['target']
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

(5700, 256, 128)
(5700, 256, 128, 3)
(5700,)


In [4]:
from IPython.display import clear_output

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()
        
plot_losses = PlotLosses()

In [5]:
# Laden des Modells
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 128, 3))

# Hinzufügen von neuen Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(19, activation='softmax')(x)

# Zusammenfügen des alten und neuen Modells
model = Model(inputs=base_model.input, outputs=predictions)

# Einfrieren der Gewichte des alten Modells
for layer in base_model.layers:
    layer.trainable = False

# Kompilieren des Modells
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training des Modells
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[plot_losses])

: 